[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/05-RGB_compression/companded.ipynb)

# RGB compression using a companded quantizer

In [ ]:
!ln -sf ~/MRVC/src/debug.py .
!ln -sf ~/MRVC/src/logging_config.py .
!ln -sf ~/repos/quantization/companded_quantizer.py .
!ln -sf ~/repos/quantization/distortion.py .
!ln -sf ~/MRVC/src/image_3.py .
!ln -sf ~/MRVC/src/image_1.py .
!ln -sf ~/repos/quantization/information.py .

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
import companded_quantizer as companded
import distortion
import image_3 as RGB_image
import image_1 as gray_image
import colored
import information

## Configuration

In [ ]:
# Prefix of the RGB image to be quantized.

home = os.environ["HOME"]
#fn = home + "/MRVC/images/stockholm/"
#fn = home + "/MRVC/images/Hommer/"
#fn = home + "/MRVC/images/fluorescence/"
#fn = home + "/MRVC/images/blue_rose/"
fn = home + "/MRVC/images/lena_color/"
#fn = home + "/MRVC/images/nored_lena/"

components = ['R', 'G', 'B']

quantizer = companded

# Maybe this does not make sense in digital quantization (generate loops in the RD domain
# because the output number of quantization indexes is not a power of 2.)
#Q_steps = range(128, 0, -4)

Q_steps = [2**i for i in range(7, -1, -1)] # Quantization steps (simulating bit-plane encoding)

#quantizer = companded
#Q_steps = [2**i for i in range(16, -1, -1)] # Quantization steps (simulating bit-plane encoding)

print(Q_steps)

RGB_image.write = RGB_image.debug_write # faster
#RGB_image.write = RGB_image.write # higher compression

gray_image.write = gray_image.debug_write # faster
#gray_image.write = gray_image.write # higher compression

Notice that non embbeded quatization (using steps thar are different from a power of 2) steps (can produce loops in the RD curves due to the non-linearity of the integer division performed in the quantization).

## Read the image and show it

In [ ]:
RGB_img = RGB_image.read(fn).astype(np.int16) # 8 bits/components is not sufficient to shift the components to [-128, 127]
print(RGB_img.dtype)
print(RGB_img.max(), RGB_img.min())
RGB_image.show(RGB_img, fn + "000.png")

In [ ]:
gray_image.show(RGB_img[..., 0], fn + " (R channel)")

In [ ]:
gray_image.show(RGB_img[..., 1], fn + " (G channel)")

In [ ]:
gray_image.show(RGB_img[..., 2], fn + " (B channel)")

## Show some quantizations

In [ ]:
QS = 128 # Quantization Step
RGB_img = RGB_img.astype(np.int16) - 128 # Quantized data must be centered at 0
y, k = quantizer.quan_dequan(RGB_img, QS)
print("Used quantization indexes:", np.unique(k))
RGB_image.show_normalized(k, f"{quantizer.name} $\\Delta={QS}$")

In [ ]:
QS = 64 # Quantization Step
RGB_img = RGB_img.astype(np.int16) - 128 # Quantized data must be centered at 0
y, k = quantizer.quan_dequan(RGB_img, QS)
print("Used quantization indexes:", np.unique(k))
RGB_image.show_normalized(k, f"{quantizer.name} $\\Delta={QS}$")

## RD curve using same $\Delta$ for each RGB channel ($\Delta_{\text{R}} = \Delta_{\text{G}} = \Delta_{\text{B}}$)
To see the contribution of each channel to the RD curve.

In [ ]:
def same_delta_RD_curve(RGB_img, Q_steps, quantizer):
    RGB_img -= 128
    points = []
    for Q_step in Q_steps:
        y, k = quantizer.quan_dequan(RGB_img, Q_step)
        k = (k + 128).astype(np.uint8) # Only positive components can be written in an PNG file
        print("Quantization indexes: ", np.unique(k))
        #rate = common.bits_per_color_pixel(k, str(Q_step) + '_') # Remember that k has 3 components
        rate = RGB_image.debug_write(k, "/tmp/" + str(Q_step) + '_', 0)*8/k.size
        _distortion = distortion.RMSE(RGB_img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

#same_delta_RD_points = same_delta_RD_curve(img - 128, Q_steps, quantizer)
same_delta_RD_points = same_delta_RD_curve(RGB_img, Q_steps, quantizer)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*same_delta_RD_points), c='m', marker='x', label=f"{quantizer.name}", linestyle="dotted")
pylab.title(f"Rate/Distortion Performance ({quantizer.name})")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
print(quantizer.name)
with open(f"RGB_same_{quantizer.name}_RD_points.txt", 'w') as f:
    for item in same_delta_RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")